In [4]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
from deep_translator import GoogleTranslator
from nanoid import generate
from slugify import slugify
from transformers import pipeline

### URL: https://www.epam.com/careers/job-listings?country=Mexico

In [ ]:
response = requests.get('https://www.epam.com/services/vacancy/search?locale=en&limit=200&recruitingUrl=%2Fcontent%2Fepam%2Fen%2Fcareers%2Fjob-listings%2Fjob&query=&country=Mexico&sort=relevance&offset=0&searchType=placeOfWorkFilter&_=1709566523032')

dictionario = response.json()

print(dictionario)

print(len(dictionario['result']))

In [ ]:
def extraer_contenido_div(url):
    # Realizamos la solicitud GET a la URL
    response = requests.get(url)
    
    # Verificamos si la solicitud fue exitosa
    if response.status_code == 200:
        # Parseamos el contenido HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontramos el div con la clase deseada
        div_contenido = soup.find('div', class_='vacancy-details-23__content-holder')  
        
        # Verificamos si se encontró el div
        if div_contenido:
            # Inicializamos una lista para almacenar el contenido
            contenido = []
            
            # Recorremos todos los elementos <h3> dentro del div
            for h3_tag in div_contenido.find_all('h3'):
                # Agregamos el texto de cada título a la lista de contenido
                contenido.append(h3_tag.text.strip())
            
            return contenido
        else:
            print("No se encontró el div con la clase especificada.")
    else:
        print("Error al obtener la página:", response.status_code)

In [ ]:
# URL de la página a scrapear
url = 'https://www.epam.com/careers/job-listings/job.95046.senior-support-project-administrator_guadalajara_mexico'

# Llamamos a la función y mostramos el contenido extraído
contenido_div = extraer_contenido_div(url)
if contenido_div:
    print("Contenido del div:")
    for item in contenido_div:
        print(item)

In [ ]:
url = 'https://www.epam.com/careers/job-listings/job.95046.senior-support-project-administrator_guadalajara_mexico'
# Parseamos el contenido HTML
soup = BeautifulSoup(url, 'html.parser')

# Encontramos el div con la clase "vacancy-details-23__content-holder"
div_contenido = soup.find('div', class_='vacancy-details-23__content-holder')

# Verificamos si se encontró el div
if div_contenido:
    # Inicializamos un diccionario para almacenar el contenido
    contenido_por_titulo = {}
    
    # Recorremos todos los elementos <h3> dentro del div
    for h3_tag in div_contenido.find_all('h3'):
        # Inicializamos una lista para almacenar los elementos de lista correspondientes a este título
        elementos_li = []
        
        # Buscamos el siguiente elemento <ul> hermano del título <h3>
        ul_tag = h3_tag.find_next_sibling('ul')
        
        # Recorremos todos los elementos <li> dentro del <ul>
        for li_tag in ul_tag.find_all('li'):
            # Agregamos el texto del elemento <li> a la lista de elementos_li
            elementos_li.append(li_tag.text.strip())
        
        # Almacenamos los elementos de lista en el diccionario bajo el título correspondiente
        contenido_por_titulo[h3_tag.text.strip()] = elementos_li

# Mostramos el contenido almacenado en variables con los nombres de los títulos
for titulo, elementos in contenido_por_titulo.items():
    # Creamos una variable con el nombre del título y le asignamos la lista de elementos
    globals()[titulo.replace(" ", "_").lower()] = elementos

# Ahora puedes acceder a cada lista de elementos utilizando el nombre de la variable correspondiente al título
print("Contenido almacenado en variables:")
print("Responsibilities:", responsibilities)
print("Requirements:", requirements)
print("Technologies:", technologies)
print("We_Offer:", we_offer)
print("Conditions:", conditions)

In [ ]:
def extraer_contenido_h3_ul_desde_url(url):
    try:
        # Realizamos la solicitud GET a la URL
        response = requests.get(url)
        
        # Verificamos si la solicitud fue exitosa
        if response.status_code == 200:
            # Parseamos el contenido HTML
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Encontramos el div con la clase "vacancy-details-23__content-holder"
            div_contenido = soup.find('div', class_='vacancy-details-23__content-holder')
            
            # Verificamos si se encontró el div
            if div_contenido:
                # Inicializamos un diccionario para almacenar el contenido
                contenido_por_titulo = {}
                
                # Recorremos todos los elementos <h3> dentro del div
                for h3_tag in div_contenido.find_all('h3'):
                    # Inicializamos una lista para almacenar los elementos de lista correspondientes a este título
                    elementos_li = []
                    
                    # Buscamos el siguiente elemento <ul> hermano del título <h3>
                    ul_tag = h3_tag.find_next_sibling('ul')
                    
                    # Recorremos todos los elementos <li> dentro del <ul>
                    for li_tag in ul_tag.find_all('li'):
                        # Agregamos el texto del elemento <li> a la lista de elementos_li
                        elementos_li.append(li_tag.text.strip())
                    
                    # Almacenamos los elementos de lista en el diccionario bajo el título correspondiente
                    contenido_por_titulo[h3_tag.text.strip()] = elementos_li
                
                return contenido_por_titulo
            else:
                print("No se encontró el div con la clase especificada.")
        else:
            print("Error al obtener la página:", response.status_code)
    except Exception as e:
        print("Ocurrió un error:", e)

In [ ]:
# Ejemplo de uso
url = 'https://www.epam.com/careers/job-listings/job.95046.senior-support-project-administrator_guadalajara_mexico'
contenido_por_titulo = extraer_contenido_h3_ul_desde_url(url)

if contenido_por_titulo:
    # Mostramos el contenido almacenado en el diccionario
    for titulo, elementos in contenido_por_titulo.items():
        print(f"{titulo}:")
        for elemento in elementos:
            print(f"- {elemento}")
        print()

In [ ]:
def epam(df: pd.DataFrame) -> pd.DataFrame:
    df["name"] = clean_title(df["name"])
    df["companyName"] = df["companyName"].replace(df["companyName"].values, "Apple")
    df["Otros_requisitos"] = df["Otros_requisitos"].fillna("")
    df["Educación_Experiencia"] = df["Educación_Experiencia"].fillna("")
    columns_to_merge = [
        "Responsabilities",
        "Requirements",
        "We Offer",
        "conditions"
    ]
    column_merge_new = "Req_txt"
    columns_to_delete = [
        "Responsabilities",
        "Requirements",
        "conditions"
    ]
    df = merge_clean_delete(df, column_merge_new, columns_to_merge, columns_to_delete)
    df["skills"] = df["Req_txt"].apply(find_skills)
    df["skills"] = df["skills"].drop_duplicates(keep="last")
    df["skills"] = df["skills"].apply(dict_skills)
    df["tools"] = df["Req_txt"].apply(find_tools)
    df["tools"] = df["tools"].apply(dict_tools)
    df["tools"] = df["tools"].drop_duplicates(keep="last")
    df["aptitudes"] = df["Req_txt"].apply(find_aptitudes)
    df["aptitudes"] = df["aptitudes"].drop_duplicates(keep="last")
    df["aptitudes"] = df["aptitudes"].apply(dict_aptitudes)
    df["languages"] = df["Req_txt"].apply(find_languages)
    df["languages"] = df["languages"].drop_duplicates(keep="last")
    df["languages"] = df["languages"].apply(dict_languages)
    df = df.drop("Req_txt", axis=1)
    df = translate_columns_deep_translator(
        df, column_names=["aptitudes", "languages", "skills"]
    )
    df = complete_dataframe(df)
    df = fill_data(df, companyId="epam")
    df["skills"] = df["skills"].apply(json_nan)
    df["tools"] = df["tools"].apply(json_nan)
    df["aptitudes"] = df["aptitudes"].apply(json_nan)
    df["languages"] = df["languages"].apply(json_nan)
    return df

In [ ]:
diccionario_epam = {
    "userId": "epam",
    "name": "epam",
    "description": """
                    EPAM Systems, Inc. es una empresa estadounidense que se especializa en servicios de ingeniería de software, ingeniería de plataformas digitales y diseño de productos digitales, que opera en Newtown, Pensilvania. EPAM es miembro fundador de la Alianza MACH.
                    """,
    "website": "www.epam.com//",
    "sector": "Tecnologías de la información y la comunicación",
    "isExternal": True,
}

df_epam = pd.DataFrame(diccionario_epam, index=[0])